
![Pytorch Cpp](https://github.com/prabhuomkar/pytorch-cpp/raw/master/images/pytorch_cpp.png)

* **This notebook is intended to be used on [Google Colab](https://colab.research.google.com) ONLY!** 
* It allows you
to build and run the [pytorch-cpp](https://github.com/prabhuomkar/pytorch-cpp) tutorials on a hosted GPU equipped system for free.

#Setup
1. Switch to a GPU runtime: `Runtime -> Change runtime type -> Hardware accelerator: Select GPU`
2. Run each of the following setup cells one after the other.

##Magics and Imports

In [ ]:
# For nicer looking images
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

# Python package imports to display images
import matplotlib.pyplot as plt
from PIL import Image
from pathlib import Path
from google.colab import widgets
from google.colab import output
import matplotlib.pyplot as plt
import pandas as pd
import os

In [ ]:
# Image plotting helper function
def plot_images(file_paths, nrows, ncols, figsize_per_image):
    fig, ax = plt.subplots(nrows, ncols, figsize=(figsize_per_image[0] * ncols, figsize_per_image[1] * nrows))

    n_files = len(file_paths)

    if nrows == 1 or ncols == 1:
        for i, image_path in enumerate(file_paths):
            ax[i].imshow(Image.open(image_path), aspect="equal")
            ax[i].axis("off")
            ax[i].set_title(image_path.name)
    else:
        for i in range(nrows):
            for j in range(ncols):
                ax[i,j].imshow(Image.open(file_paths[i * ncols + j]), aspect="equal")
                ax[i,j].axis("off")
                ax[i,j].set_title(file_paths[i * ncols + j].name)
    fig.tight_layout()
    plt.show()

# Image captioning helper functions
def create_output_datafram(output_path):
    output_data = []
    for f in output_path.iterdir():
        img_name, num, word = f.name.rsplit('_', 2)
        word = word.split('.')[0]
        output_data.append({"file": f, "image": img_name, "num": int(num), "word": word})

    df = pd.DataFrame(output_data)

    return df.groupby("image").apply(pd.DataFrame.sort_values, 'num')[["file", "word"]].groupby("image").agg(list)

def create_tab(output_example):
    print(f"Image: {output_example.Index}")

    t = widgets.TabBar(output_example.word)

    for i, image_file in enumerate(output_example.file):
        with t.output_to(i, select=False):
            _, ax = plt.subplots(figsize=(8, 8))
            ax.imshow(Image.open(image_file), aspect="equal")
            ax.axis("off")
            plt.show()

def plot_image_caption_output(output_path):
    df = create_output_datafram(output_path)
    for p, _ in zip(df.itertuples(), widgets.Grid(len(df), 1)):
        create_tab(p)

##Install more recent versions of dependencies and build tools
(This takes several minutes)

In [ ]:
# Update CUDA 
!wget https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64/cuda-ubuntu1804.pin
!sudo mv cuda-ubuntu1804.pin /etc/apt/preferences.d/cuda-repository-pin-600
!wget https://developer.download.nvidia.com/compute/cuda/11.6.0/local_installers/cuda-repo-ubuntu1804-11-6-local_11.6.0-510.39.01-1_amd64.deb
!sudo dpkg -i 'cuda-repo-ubuntu1804-11-6-local_11.6.0-510.39.01-1_amd64.deb'
!sudo apt-key add /var/cuda-repo-ubuntu1804-11-6-local/7fa2af80.pub
!sudo apt-get update
!sudo DEBIAN_FRONTEND=noninteractive apt-get -y install cuda-11-6
!sudo cp /usr/include/x86_64-linux-gnu/cudnn*.h /usr/local/cuda-11.6/include
!sudo cp /usr/lib/x86_64-linux-gnu/libcudnn* /usr/local/cuda-11.6/lib64
!sudo chmod a+r /usr/local/cuda-11.6/include/cudnn*.h /usr/local/cuda-11.6/lib64/libcudnn*
old_ld_library_path = %env LD_LIBRARY_PATH
new_ld_library_path = "/usr/local/cuda-11.6/lib64:" + old_ld_library_path
%env LD_LIBRARY_PATH={new_ld_library_path}
%env CUDA_HOME=/usr/local/cuda-11.6

# Update CMake
!rm -rf deps
!mkdir deps
%cd /content/deps
!wget --no-check-certificate https://www.cmake.org/files/v3.19/cmake-3.19.0-Linux-x86_64.tar.gz
!tar -xvf cmake-3.19.0-Linux-x86_64.tar.gz > /dev/null
%mv cmake-3.19.0-Linux-x86_64 cmake-install
old_path = %env PATH
new_path = "/content/deps/cmake-install:/content/deps/cmake-install/bin:" + old_path
%env PATH={new_path}
%cd /content/
!cmake --version

# Update g++
!sudo add-apt-repository ppa:ubuntu-toolchain-r/test -y
!sudo apt-get update
!sudo apt-get install gcc-10 g++-10 -y
!sudo update-alternatives --install /usr/bin/gcc gcc /usr/bin/gcc-10 60 --slave /usr/bin/g++ g++ /usr/bin/g++-10

##Installed Programs & GPU

In [ ]:
%%bash
# Preinstalled compiler versions
g++ --version
gcc --version
# Check installed CMake version
cmake --version
echo ""
# Preinstalled pytorch/torchvision versions
pip show torch
echo ""
pip show torchvision
echo ""
# CUDA Information
nvcc --version

##Clone Repo

In [ ]:
!git clone https://github.com/prabhuomkar/pytorch-cpp.git
%cd /content/pytorch-cpp
%ls

##Generate Build System

In [ ]:
%cd /content/pytorch-cpp
%rm -rf build
!cmake -B build -D CUDA_V=11.8 -D CMAKE_BUILD_TYPE=Release

##Build Tutorials

In [ ]:
%cd /content/pytorch-cpp
!cmake --build build

#Run Tutorials

### 1. Basics
* [PyTorch Basics](#scrollTo=OVBcBgknvm_W)
* [Linear Regression](#scrollTo=eX6YN9DB04r2)
* [Logistic Regression](#scrollTo=3rfrwqKU1VZm)
* [Feedforward Neural Network](#scrollTo=ywmM2xkt1mi0)

### 2. Intermediate
* [Convolutional Neural Network](#scrollTo=Convolutional_Neural_Network)
* [Deep Residual Network](#scrollTo=Kla2SaVv228f)
* [Recurrent Neural Network](#scrollTo=aTrOcUke3Zxu)
* [Bidirectional Recurrent Neural Network](#scrollTo=FVVx0XFY3yvU)
* [Language Model (RNN-LM)](#scrollTo=gnvBsukO4H00)

### 3. Advanced
* [Generative Adversarial Networks](#scrollTo=ggBdsvyR8UMN)
* [Variational Auto-Encoder](#scrollTo=CYzbCRPo_TJ7)
* [Neural Style Transfer](#scrollTo=ix6R5AosZHTx)
* [Image Captioning (CNN-RNN)](#scrollTo=aEBcqiKr-5Sb)

##Basics

In [ ]:
%ls /content/pytorch-cpp/build/tutorials/basics/ -1

### Pytorch-Basics

In [ ]:
# Create required torchscript module:
%cd /content/pytorch-cpp/tutorials/basics/pytorch_basics/model/
!python create_resnet18_scriptmodule.py

In [ ]:
%ls /content/pytorch-cpp/build/tutorials/basics/pytorch_basics/

In [ ]:
# Run
%cd /content/pytorch-cpp/build/tutorials/basics/pytorch_basics/
!./pytorch-basics

###Linear Regression

In [ ]:
%ls /content/pytorch-cpp/build/tutorials/basics/linear_regression/

In [ ]:
# Run
%cd /content/pytorch-cpp/build/tutorials/basics/linear_regression/
!./linear-regression

###Logistic Regression

In [ ]:
%ls /content/pytorch-cpp/build/tutorials/basics/logistic_regression/

In [ ]:
# Run
%cd /content/pytorch-cpp/build/tutorials/basics/logistic_regression/
!./logistic-regression

###Feedforward Neural Network

In [ ]:
%ls /content/pytorch-cpp/build/tutorials/basics/feedforward_neural_network/

In [ ]:
# Run
%cd /content/pytorch-cpp/build/tutorials/basics/feedforward_neural_network/
!./feedforward-neural-network

##Intermediate

In [ ]:
%ls /content/pytorch-cpp/build/tutorials/intermediate/ -1

###Convolutional Neural Network

In [ ]:
%ls /content/pytorch-cpp/build/tutorials/intermediate/convolutional_neural_network/

In [ ]:
# Run
%%time
%cd /content/pytorch-cpp/build/tutorials/intermediate/convolutional_neural_network/
!./convolutional-neural-network

###Deep Residual Network

In [ ]:
%ls /content/pytorch-cpp/build/tutorials/intermediate/deep_residual_network/

In [ ]:
# Run
%%time
%cd /content/pytorch-cpp/build/tutorials/intermediate/deep_residual_network/
!./deep-residual-network

###Recurrent Neural Network

In [ ]:
%ls /content/pytorch-cpp/build/tutorials/intermediate/recurrent_neural_network/

In [ ]:
# Run
%%time
%cd /content/pytorch-cpp/build/tutorials/intermediate/recurrent_neural_network/
!./recurrent-neural-network

###Bidirectional Recurrent Neural Network

In [ ]:
%ls /content/pytorch-cpp/build/tutorials/intermediate/bidirectional_recurrent_neural_network/

In [ ]:
# Run
%%time
%cd /content/pytorch-cpp/build/tutorials/intermediate/bidirectional_recurrent_neural_network/
!./bidirectional-recurrent-neural-network

###Language Model


In [ ]:
%ls /content/pytorch-cpp/build/tutorials/intermediate/language_model/

In [ ]:
# Run
%%time
%cd /content/pytorch-cpp/build/tutorials/intermediate/language_model/
!./language-model

In [ ]:
# Results
%cat /content/pytorch-cpp/build/tutorials/intermediate/language_model/output/sample.txt

##Advanced

In [ ]:
%ls /content/pytorch-cpp/build/tutorials/advanced/

###Generative Adversarial Network

In [ ]:
%ls /content/pytorch-cpp/build/tutorials/advanced/generative_adversarial_network/

In [ ]:
# Run
%%time
%cd /content/pytorch-cpp/build/tutorials/advanced/generative_adversarial_network/
!./generative-adversarial-network

In [ ]:
# Results
%ls /content/pytorch-cpp/build/tutorials/advanced/generative_adversarial_network/output/ -1 -r --sort=time | head -10
!echo "..."
%ls /content/pytorch-cpp/build/tutorials/advanced/generative_adversarial_network/output/ -1 -r --sort=time | tail -10

In [ ]:
# Show results:
# Get paths of output files sorted by creation time:
gan_output_file_paths = sorted(list(Path("/content/pytorch-cpp/build/tutorials/advanced/generative_adversarial_network/output").iterdir()), key=os.path.getmtime)
# Select some indices to show
gan_display_indices = [0, 1, len(gan_output_file_paths) // 2, len(gan_output_file_paths) - 1]
gan_file_paths = [gan_output_file_paths[i] for i in gan_display_indices]

plot_images(gan_file_paths, nrows=1, ncols=len(gan_file_paths), figsize_per_image=(5, 5))

###Variational Autoencoder

In [ ]:
%ls /content/pytorch-cpp/build/tutorials/advanced/variational_autoencoder/

In [ ]:
# Run
%%time
%cd /content/pytorch-cpp/build/tutorials/advanced/variational_autoencoder/
!./variational-autoencoder

In [ ]:
# Results
%ls /content/pytorch-cpp/build/tutorials/advanced/variational_autoencoder/output/ -1 -r --sort=time | head -10
!echo "..."
%ls /content/pytorch-cpp/build/tutorials/advanced/variational_autoencoder/output/ -1 -r --sort=time | tail -10

In [ ]:
vae_output_file_paths = sorted(list(Path("./output").iterdir()), key=os.path.getmtime)
vae_display_indices = [1, 0, len(vae_output_file_paths) // 2, len(vae_output_file_paths) // 2 - 1, len(vae_output_file_paths) - 1, len(vae_output_file_paths) - 2]
vae_file_paths = [vae_output_file_paths[i] for i in vae_display_indices]

plot_images(vae_file_paths, nrows=len(vae_file_paths) // 2, ncols=2, figsize_per_image=(9, 5))

###Neural Style Transfer

In [ ]:
# Create required torchscript module:
%cd /content/pytorch-cpp/tutorials/advanced/neural_style_transfer/model/
!python create_vgg19_layers_scriptmodule.py

In [ ]:
%ls /content/pytorch-cpp/build/tutorials/advanced/neural_style_transfer/

In [ ]:
# Run
%%time
%cd /content/pytorch-cpp/build/tutorials/advanced/neural_style_transfer/
!./neural-style-transfer

In [ ]:
# Results
%ls /content/pytorch-cpp/build/tutorials/advanced/neural_style_transfer/output/ -1 -r --sort=time

In [ ]:
# Inputs
%ls /content/pytorch-cpp/data/neural_style_transfer_images/

In [ ]:
nst_input_file_paths = sorted(list(Path("/content/pytorch-cpp/data/neural_style_transfer_images").iterdir()))

plot_images(nst_input_file_paths, nrows=1, ncols=len(nst_input_file_paths), figsize_per_image=(8, 5))

In [ ]:
nst_output_file_paths = sorted(list(Path("/content/pytorch-cpp/build/tutorials/advanced/neural_style_transfer/output").iterdir()), key=os.path.getmtime)

plot_images(nst_output_file_paths, nrows=len(nst_output_file_paths), ncols=1, figsize_per_image=(7, 5))

###Image Captioning

In [ ]:
# Create required torchscript module:
%cd /content/pytorch-cpp/tutorials/advanced/image_captioning/model/
!python create_encoder_cnn_backbone_scriptmodule.py

In [ ]:
%ls /content/pytorch-cpp/build/tutorials/advanced/image_captioning/

In [ ]:
# Show command line arguments:
%cd /content/pytorch-cpp/build/tutorials/advanced/image_captioning/
!./image-captioning --help

In [ ]:
# Run
%%time
%cd /content/pytorch-cpp/build/tutorials/advanced/image_captioning/
!./image-captioning --batch_size=128 --num_epochs=4

In [ ]:
# Results
plot_image_caption_output(Path("/content/pytorch-cpp/build/tutorials/advanced/image_captioning/output"))